<h2 style="color:#1f77b4;">Live Coding Parte 1</h2>

<h5 style="color:#1f77b4;">1. Importa la librería pandas</h5>

In [123]:
import pandas as pd

<h5 style="color:#1f77b4;">2. Elegir una página web con tablas HTML públicas (ej: Wikipedia)</h5>

In [124]:
url = "https://en.wikipedia.org/wiki/List_of_largest_cities"

<h5 style="color:#1f77b4;">3. Usar read_html() para extraer todas las tablas</h5>

In [88]:
#pip install requests


Note: you may need to restart the kernel to use updated packages.


In [125]:
import pandas as pd
import requests

url = "https://en.wikipedia.org/wiki/List_of_largest_cities"

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0 Safari/537.36"
}

resp = requests.get(url, headers=headers, timeout=30)
resp.raise_for_status() 

tablas = pd.read_html(resp.text)
print("Tablas encontradas:", len(tablas))


Tablas encontradas: 7


/var/folders/4r/mkm_qnw15rq9j0mrls43rqy00000gn/T/ipykernel_6677/1524441047.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tablas = pd.read_html(resp.text)


<h5 style="color:#1f77b4;">4.Seleccionar una tabla por índice</h5>

In [126]:
for i, t in enumerate(tablas[:10]):  # muestra solo las primeras 10
    print(f"Tabla {i}: {t.shape}")

Tabla 0: (10, 1)
Tabla 1: (83, 13)
Tabla 2: (6, 2)
Tabla 3: (2, 6)
Tabla 4: (1, 6)
Tabla 5: (8, 2)
Tabla 6: (9, 2)


In [127]:
df = tablas[1].copy()

In [128]:
df.columns

MultiIndex([(                         'City[a]', ...),
            (                         'Country', ...),
            ('UN 2025 population estimates[11]', ...),
            (                  'City proper[b]', ...),
            (                  'City proper[b]', ...),
            (                  'City proper[b]', ...),
            (                  'City proper[b]', ...),
            (                  'Urban area[12]', ...),
            (                  'Urban area[12]', ...),
            (                  'Urban area[12]', ...),
            (            'Metropolitan area[c]', ...),
            (            'Metropolitan area[c]', ...),
            (            'Metropolitan area[c]', ...)],
           )

In [129]:
df.head()

City[a]     Country UN 2025 population estimates[11]  \
    City[a]     Country UN 2025 population estimates[11]   
0   Jakarta   Indonesia                         41913860   
1     Dhaka  Bangladesh                         36585479   
2     Tokyo       Japan                         33412512   
3     Delhi       India                         30222405   
4  Shanghai       China                         29558908   

               City proper[b]                                         \
                   Definition Population Area (km2)   Density (/km2)   
0              Special region   10154134        664      15,292 [13]   
1                Capital city   10295407        338  30,460 [15][16]   
2       Metropolis prefecture   13515271       2191       6,169 [18]   
3  National Capital Territory   16753235       1484      11,289 [20]   
4                Municipality   24870895       6341   3,922 [22][23]   

  Urban area[12]                           Metropolitan area[c]             \
      Population Area (km2) Density (/km2)           Population Area (km2)   
0       33756000       3546      9,519 [d]             33430285       7063   
1       19134000        619          30911      14,543,124 [17]       —N/a   
2       37785000       8231      4,591 [e]             37274000      13452   
3       32226000       2344     13,748 [f]             29000000       3483   
4       24073000       4333      5,556 [g]                 —N/a       —N/a   

                  
  Density (/km2)  
0     4,733 [14]  
1           —N/a  
2     2,771 [19]  
3     8,326 [21]  
4           —N/a

<h5 style="color:#1f77b4;">5. Renombrar columnas de forma explícita.</h5>

1. Recorre columna por columna
for col in df.columns
donde col es una tupla tipo ("Urban area[12]", "Population").

2. Convierte cada parte a texto y elimina las partes vacías (nan)
[str(x) for x in col if str(x) != "nan"]
A veces pandas pone nan cuando hay celdas de encabezado vacías en alguna fila.
Esto las saca para que no te quede “nan Population”.

3. Une las partes con un espacio y limpia bordes
" ".join(...).strip()

In [130]:
# 5) Aplanar columnas si vienen en MultiIndex
if isinstance(df.columns, pd.MultiIndex):   # Detecta si las columnas vienen “dobles”
    df.columns = [
        " ".join([str(x) for x in col if str(x) != "nan"]).strip() 
        for col in df.columns               # Recorre columna por columna
    ]

In [131]:
df.columns

Index(['City[a] City[a]', 'Country Country',
       'UN 2025 population estimates[11] UN 2025 population estimates[11]',
       'City proper[b] Definition', 'City proper[b] Population',
       'City proper[b] Area (km2)', 'City proper[b] Density (/km2)',
       'Urban area[12] Population', 'Urban area[12] Area (km2)',
       'Urban area[12] Density (/km2)', 'Metropolitan area[c] Population',
       'Metropolitan area[c] Area (km2)',
       'Metropolitan area[c] Density (/km2)'],
      dtype='object')

In [132]:
df.head()

,City[a] City[a],Country Country,UN 2025 population estimates[11] UN 2025 population estimates[11],City proper[b] Definition,City proper[b] Population,City proper[b] Area (km2),City proper[b] Density (/km2),Urban area[12] Population,Urban area[12] Area (km2),Urban area[12] Density (/km2),Metropolitan area[c] Population,Metropolitan area[c] Area (km2),Metropolitan area[c] Density (/km2)
0,Jakarta,Indonesia,41913860,Special region,10154134,664,"15,292 [13]",33756000,3546,"9,519 [d]",33430285,7063,"4,733 [14]"
1,Dhaka,Bangladesh,36585479,Capital city,10295407,338,"30,460 [15][16]",19134000,619,30911,"14,543,124 [17]",—N/a,—N/a
2,Tokyo,Japan,33412512,Metropolis prefecture,13515271,2191,"6,169 [18]",37785000,8231,"4,591 [e]",37274000,13452,"2,771 [19]"
3,Delhi,India,30222405,National Capital Territory,16753235,1484,"11,289 [20]",32226000,2344,"13,748 [f]",29000000,3483,"8,326 [21]"
4,Shanghai,China,29558908,Municipality,24870895,6341,"3,922 [22][23]",24073000,4333,"5,556 [g]",—N/a,—N/a,—N/a


In [133]:
# Renombrar explícitamente algunas claves (para que sea más legible)
df = df.rename(columns={
    "City[a] City[a]": "City",
    "Country Country": "Country",
    "UN 2025 population estimates[11] UN 2025 population estimates[11]": "UN_2025_pop",
    "City proper[b] Population": "City_proper_pop",
    "Urban area[12] Population": "Urban_pop",
    "City proper[b] Definition" : "City:prop_def",
    "Metropolitan area[c] Population": "Metro_pop",
})

In [134]:
df.head()

,City,Country,UN_2025_pop,City:prop_def,City_proper_pop,City proper[b] Area (km2),City proper[b] Density (/km2),Urban_pop,Urban area[12] Area (km2),Urban area[12] Density (/km2),Metro_pop,Metropolitan area[c] Area (km2),Metropolitan area[c] Density (/km2)
0,Jakarta,Indonesia,41913860,Special region,10154134,664,"15,292 [13]",33756000,3546,"9,519 [d]",33430285,7063,"4,733 [14]"
1,Dhaka,Bangladesh,36585479,Capital city,10295407,338,"30,460 [15][16]",19134000,619,30911,"14,543,124 [17]",—N/a,—N/a
2,Tokyo,Japan,33412512,Metropolis prefecture,13515271,2191,"6,169 [18]",37785000,8231,"4,591 [e]",37274000,13452,"2,771 [19]"
3,Delhi,India,30222405,National Capital Territory,16753235,1484,"11,289 [20]",32226000,2344,"13,748 [f]",29000000,3483,"8,326 [21]"
4,Shanghai,China,29558908,Municipality,24870895,6341,"3,922 [22][23]",24073000,4333,"5,556 [g]",—N/a,—N/a,—N/a


<h5 style="color:#1f77b4;">6. Limpiar datos</h5>

In [135]:
# 6) Limpieza y conversión de tipos
# - quitar referencias tipo [13] en toda la tabla
df = df.replace(r"\[.*?\]", "", regex=True)

In [136]:
df.head()

,City,Country,UN_2025_pop,City:prop_def,City_proper_pop,City proper[b] Area (km2),City proper[b] Density (/km2),Urban_pop,Urban area[12] Area (km2),Urban area[12] Density (/km2),Metro_pop,Metropolitan area[c] Area (km2),Metropolitan area[c] Density (/km2)
0,Jakarta,Indonesia,41913860,Special region,10154134,664,"15,292",33756000,3546,"9,519",33430285,7063,"4,733"
1,Dhaka,Bangladesh,36585479,Capital city,10295407,338,"30,460",19134000,619,30911,"14,543,124",—N/a,—N/a
2,Tokyo,Japan,33412512,Metropolis prefecture,13515271,2191,"6,169",37785000,8231,"4,591",37274000,13452,"2,771"
3,Delhi,India,30222405,National Capital Territory,16753235,1484,"11,289",32226000,2344,"13,748",29000000,3483,"8,326"
4,Shanghai,China,29558908,Municipality,24870895,6341,"3,922",24073000,4333,"5,556",—N/a,—N/a,—N/a


In [137]:
# - convertir columnas numéricas (población/área/densidad) eliminando comas y guiones raros
def to_number(s):
    return pd.to_numeric(
        s.astype(str)
         .str.replace(",", "", regex=False)
         .str.replace("—", "", regex=False)
         .str.replace("–", "", regex=False)
         .str.strip(),
        errors="coerce"
    )

for c in df.columns:
    if any(k in c.lower() for k in ["pop", "population", "area", "density"]):
        df[c] = to_number(df[c])

# - eliminar filas “vacías” (por ejemplo, donde no hay ciudad)
df = df.dropna(subset=["City"])

In [138]:
df.head()

,City,Country,UN_2025_pop,City:prop_def,City_proper_pop,City proper[b] Area (km2),City proper[b] Density (/km2),Urban_pop,Urban area[12] Area (km2),Urban area[12] Density (/km2),Metro_pop,Metropolitan area[c] Area (km2),Metropolitan area[c] Density (/km2)
0,Jakarta,Indonesia,41913860,Special region,10154134.0,664.0,15292.0,33756000.0,3546.0,9519.0,33430285.0,7063.0,4733.0
1,Dhaka,Bangladesh,36585479,Capital city,10295407.0,338.0,30460.0,19134000.0,619.0,30911.0,14543124.0,NaN,NaN
2,Tokyo,Japan,33412512,Metropolis prefecture,13515271.0,2191.0,6169.0,37785000.0,8231.0,4591.0,37274000.0,13452.0,2771.0
3,Delhi,India,30222405,National Capital Territory,16753235.0,1484.0,11289.0,32226000.0,2344.0,13748.0,29000000.0,3483.0,8326.0
4,Shanghai,China,29558908,Municipality,24870895.0,6341.0,3922.0,24073000.0,4333.0,5556.0,NaN,NaN,NaN


<h5 style="color:#1f77b4;">7. Guardar el resultado en un archivo CSV con to_csv().</h5>

In [ ]:
df.to_csv("largest_cities_clean.csv", index=False, encoding="utf-8")


: 